In [89]:


def readFile(inputFile):
    fileobj = open(inputFile, "r")
    a = list(fileobj)
    numNames = int(a[0].split()[0])
    numConstraints = int(a[1].split()[0])
    constraints = []
    for i in a[2:]:
        constraints.append(i.split()[:3])
    #print(constraints)
    return(makeOutsidePairs(numNames, constraints))
    
def alphaTup(string1, string2):
    return tuple([min(string1, string2), max(string1, string2)])

def makeOutsidePairs(numNames, constraints):
    nameSet = set()
    outerSet = set()
    for constraint in constraints:
        for element in constraint:
            nameSet.add(element)
        if len(nameSet) == numNames:
            break
    for name1 in nameSet:
        for name2 in nameSet:
            outerSet.add(alphaTup(name1, name2))
    print(len(outerSet))
    for constraint in constraints:
        a = alphaTup(constraint[0], constraint[1])
        if outerSet.__contains__(a):
            outerSet.remove(a)
    numAppears = {}
    for name in nameSet:
        numAppears[name] = 0
    for constraint in constraints:
        numAppears[constraint[0]] += 1 
        numAppears[constraint[1]] += 1
    for 
    print(len(outerSet))
    
        



In [90]:
readFile("phase2_inputs/inputs20/input20_0.in")

210
56
